# Projeto NLP: Análise de Sentimentos em Reviews de E-commerce

**Autor:** Fernando De Bernardi Camargo
**Data:** Setembro de 2025

---

## 1. Objetivo do Negócio

O objetivo deste projeto é construir um modelo de Machine Learning capaz de analisar o texto de avaliações de clientes e classificá-lo automaticamente como **positivo ou negativo**. Esta ferramenta permite que uma empresa de e-commerce possa, em grande escala, entender a satisfação do cliente, identificar produtos problemáticos e direcionar o atendimento de forma mais eficaz.

## 2. Carga e Preparação Inicial dos Dados

A primeira etapa consiste em carregar o dataset, que contém os textos das avaliações e suas respectivas notas, e fazer uma limpeza inicial, tratando valores nulos e selecionando as colunas relevantes para o estudo.

In [11]:
import pandas as pd

caminho_absoluto = r'C:\Users\b3rna\Documents\Projetos\analise_sentimentos\analise-de-sentimentos-ecommerce\olist_order_reviews_dataset.csv'

# Agora, carregue o arquivo usando essa variável
df_reviews = pd.read_csv(caminho_absoluto)

# Exibindo as 5 primeiras linhas do dataframe para entender a estrutura
print("As 5 primeiras linhas do nosso dataset:")
print(df_reviews.head())

# Exibindo informações gerais sobre o dataset (tipos de dados, valores nulos, etc.)
print("\nInformações sobre o dataset:")
df_reviews.info()

# Verificando a distribuição das notas (review_score)
print("\nDistribuição das notas de review:")
print(df_reviews['review_score'].value_counts())

As 5 primeiras linhas do nosso dataset:
                          review_id                          order_id  \
0  7bc2406110b926393aa56f80a40eba40  73fc7af87114b39712e6da79b0a377eb   
1  80e641a11e56f04c1ad469d5645fdfde  a548910a1c6147796b98fdf73dbeba33   
2  228ce5500dc1d8e020d8d1322874b6f0  f9e4b658b201a9f2ecdecbb34bed034b   
3  e64fb393e7b32834bb789ff8bb30750e  658677c97b385a9be170737859d3511b   
4  f7c4243c7fe1938f181bec41a392bdeb  8e6bfb81e283fa7e4f11123a3fb894f1   

   review_score review_comment_title  \
0             4                  NaN   
1             5                  NaN   
2             5                  NaN   
3             5                  NaN   
4             5                  NaN   

                              review_comment_message review_creation_date  \
0                                                NaN  2018-01-18 00:00:00   
1                                                NaN  2018-03-10 00:00:00   
2                                                

## 3. Análise Exploratória e Pré-processamento do Texto

Antes de treinar um modelo, o texto precisa ser limpo e transformado em um formato numérico.

### 3.1 Limpeza do Texto (Text Cleaning)
Nesta etapa, aplicamos uma série de transformações nos textos das avaliações para padronizá-los, incluindo:
- Conversão para letras minúsculas.
- Remoção de pontuação e caracteres especiais.
- Remoção de "stopwords" (palavras comuns como 'e', 'ou', 'de', que não agregam valor semântico).

In [12]:
# Removendo as linhas onde não há comentário de texto.
# O .dropna() remove linhas com valores nulos (NaN) na coluna especificada.
df_limpo = df_reviews.dropna(subset=['review_comment_message'])

# 2. Criando a Coluna "sentimento"
def classificar_sentimento(nota):
    if nota >= 4:
        return 'positivo'
    elif nota == 3:
        return 'neutro'
    else: # notas 1 e 2
        return 'negativo'

# Aplicando a função na coluna 'review_score' para criar a nova coluna 'sentimento'
# Usando .loc para evitar aquele aviso de "SettingWithCopyWarning"
df_limpo.loc[:, 'sentimento'] = df_limpo['review_score'].apply(classificar_sentimento)

# 3. Selecionando as Colunas Relevantes
# Criando um DataFrame final apenas com o texto e o sentimento.
df_final = df_limpo[['review_comment_message', 'sentimento']].copy()
df_final.rename(columns={'review_comment_message': 'texto'}, inplace=True)

# Exibindo as 5 primeiras linhas do nosso novo DataFrame para confirmar
print("DataFrame limpo e pronto para análise:")
print(df_final.head())

# Verificando a nova distribuição de sentimentos
print("\nNova distribuição de sentimentos:")
print(df_final['sentimento'].value_counts())

DataFrame limpo e pronto para análise:
                                                texto sentimento
3               Recebi bem antes do prazo estipulado.   positivo
4   Parabéns lojas lannister adorei comprar pela I...   positivo
9   aparelho eficiente. no site a marca do aparelh...   positivo
12    Mas um pouco ,travando...pelo valor ta Boa.\r\n   positivo
15  Vendedor confiável, produto ok e entrega antes...   positivo

Nova distribuição de sentimentos:
sentimento
positivo    26530
negativo    10890
neutro       3557
Name: count, dtype: int64


C:\Users\b3rna\AppData\Local\Temp\ipykernel_4956\2813131138.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpo.loc[:, 'sentimento'] = df_limpo['review_score'].apply(classificar_sentimento)


In [13]:
import nltk
import string

# Baixando a lista de stopwords em português (só precisa rodar uma vez)
nltk.download('stopwords')

# Pegando a lista de stopwords
stopwords = nltk.corpus.stopwords.words('portuguese')

# Criando uma função para limpar o texto
def limpar_texto(texto):
    # 1. Remove pontuação
    texto_sem_pontuacao = ''.join([char for char in texto if char not in string.punctuation])
    
    # 2. Converte para minúsculas e divide em palavras
    palavras = texto_sem_pontuacao.lower().split()
    
    # 3. Remove stopwords
    palavras_sem_stopwords = [palavra for palavra in palavras if palavra not in stopwords]
    
    # 4. Junta as palavras de volta em uma string
    return ' '.join(palavras_sem_stopwords)

# Vendo um exemplo antes e depois
print("TEXTO ORIGINAL:")
print(df_final['texto'].iloc[1]) # Pega o segundo texto do nosso dataframe

print("\nTEXTO LIMPO:")
print(limpar_texto(df_final['texto'].iloc[1]))

# Apricando a função de limpeza a toda a coluna 'texto'
df_final['texto_limpo'] = df_final['texto'].apply(limpar_texto)

# Exibindo o resultado final com a nova coluna
print("\nDataFrame com a coluna de texto limpo:")
print(df_final.head())

TEXTO ORIGINAL:
Parabéns lojas lannister adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa

TEXTO LIMPO:
parabéns lojas lannister adorei comprar internet seguro prático parabéns todos feliz páscoa


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\b3rna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



DataFrame com a coluna de texto limpo:
                                                texto sentimento  \
3               Recebi bem antes do prazo estipulado.   positivo   
4   Parabéns lojas lannister adorei comprar pela I...   positivo   
9   aparelho eficiente. no site a marca do aparelh...   positivo   
12    Mas um pouco ,travando...pelo valor ta Boa.\r\n   positivo   
15  Vendedor confiável, produto ok e entrega antes...   positivo   

                                          texto_limpo  
3                   recebi bem antes prazo estipulado  
4   parabéns lojas lannister adorei comprar intern...  
9   aparelho eficiente site marca aparelho impress...  
12                    pouco travandopelo valor ta boa  
15  vendedor confiável produto ok entrega antes prazo  


## 4. Modelagem e Treinamento

Com os dados vetorizados, dividimos o dataset em conjuntos de treino e teste. Foram treinados e avaliados diferentes algoritmos de classificação para encontrar o de melhor performance.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score, classification_report

X = df_final['texto_limpo']
y = df_final['sentimento']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# A vetorização com N-grams continua a mesma
vectorizer_ngram = TfidfVectorizer(ngram_range=(1, 2))
X_train_vec = vectorizer_ngram.fit_transform(X_train)
X_test_vec = vectorizer_ngram.transform(X_test)

# Usando max_iter=1000 para garantir que o modelo tenha tempo de convergir
# Usando class_weight='balanced' para forçar o modelo a prestar atenção nas classes minoritárias
model_lr = LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced')
model_lr.fit(X_train_vec, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,42
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


## 5. Avaliação do Modelo

O modelo final foi avaliado com base em sua acurácia na classificação de reviews que nunca havia visto antes.

In [15]:
# 5. Fazer Previsões e Avaliar o modelo de Regressão Logística
y_pred_lr = model_lr.predict(X_test_vec)

# Calcular e imprimir a acurácia
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print(f"Acurácia do modelo (Regressão Logística): {accuracy_lr * 100:.2f}%")

# Imprimir o relatório de classificação detalhado
print("\nRelatório de Classificação (Regressão Logística):")
print(classification_report(y_test, y_pred_lr))

Acurácia do modelo (Regressão Logística): 79.25%

Relatório de Classificação (Regressão Logística):
              precision    recall  f1-score   support

    negativo       0.75      0.84      0.79      2151
      neutro       0.24      0.33      0.28       730
    positivo       0.94      0.84      0.88      5315

    accuracy                           0.79      8196
   macro avg       0.64      0.67      0.65      8196
weighted avg       0.82      0.79      0.81      8196



## 6. Conclusão

O projeto demonstrou com sucesso a viabilidade de utilizar técnicas de NLP para automatizar a análise de sentimentos em um grande volume de dados textuais. O modelo desenvolvido pode ser uma ferramenta valiosa para a tomada de decisões estratégicas focadas na experiência do cliente.